<h1>Testes com os dados do Tilecal</h1>

Leitura dos dados do arquivo texto para um array de arrays em python.

In [4]:
# leitura do arquivo
with open('./data/pulses_uni0-1024_and_noise_zbEBAch01mu40.txt', 'r') as file:
  tilecal_data = [[int(value) for value in line.split()] for line in file.readlines()]

print("Arquivo contendo {} linhas com {} valores por linha".format(len(tilecal_data), len(tilecal_data[0])))

Arquivo contendo 304321 linhas com 8 valores por linha


Normalizando e separando dataset entre treino e teste: 70% treino, 30% teste

In [0]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

np_arr = np.array(tilecal_data)
X, y = np_arr[:, :7].astype('float32'), np_arr[:, 7].astype('float32')

X_scaler = StandardScaler().fit(X)
y_scaler = StandardScaler().fit(y.reshape(len(y), 1))
X = X_scaler.transform(X)
y = y_scaler.transform(y.reshape(len(y), 1))[:,0]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

Construindo, treinando e avaliando o modelo

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD

model = Sequential()
model.add(Dense(32, activation='relu', input_dim=7))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))

opt = SGD(lr=0.01, momentum=0.9)

model.compile(optimizer=opt,
              loss='mean_squared_error',
              metrics=['mse', 'mae', 'mape', 'cosine'])

history = model.fit(X_train, y_train,
          batch_size=50, epochs=5,
          verbose=1, validation_data=(X_test, y_test))

score = model.evaluate(X_test, y_test, verbose=0)

print(score)

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 213024 samples, validate on 91297 samples
Epoch 1/5
213024/213024 [==============================] - 6s 27us/step - loss: 0.0107 - mean_squared_error: 0.0107 - mean_absolute_error: 0.0682 - mean_absolute_percentage_error: 36.3802 - cosine_proximity: -0.9589 - val_loss: 0.0077 - val_mean_squared_error: 0.0077 - val_mean_absolute_error: 0.0597 - val_mean_absolute_percentage_error: 30.0963 - val_cosine_proximity: -0.9639
Epoch 2/5
213024/213024 [==============================] - 5s 23us/step - loss: 0.0068 - mean_squared_error: 0.0068 - mean_absolute_error: 0.0569 - mean_absolute_percentage_error: 30.2010 - cosine_proximity: -0.9663 - val_loss: 0.0059 - val_mean_squared_error: 0.0059 - val_mean_absolute_error: 0.0533 - val_mean_absolute_percentage_error: 25.9077 - val_cosine_proximity: -0.9684
Epoch 3/5
213024/213024 [==============================] - 5s 23us/st